# ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਨਾਲ OpenBnB MCP ਸਰਵਰ ਇੰਟੀਗ੍ਰੇਸ਼ਨ

ਇਹ ਨੋਟਬੁੱਕ ਦਿਖਾਉਂਦੀ ਹੈ ਕਿ ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਨੂੰ ਅਸਲ OpenBnB MCP ਸਰਵਰ ਨਾਲ ਕਿਵੇਂ ਵਰਤਿਆ ਜਾ ਸਕਦਾ ਹੈ, ਤਾਂ ਜੋ MCPStdioPlugin ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਸਲ Airbnb ਰਿਹਾਇਸ਼ਾਂ ਦੀ ਖੋਜ ਕੀਤੀ ਜਾ ਸਕੇ। LLM ਐਕਸੈਸ ਲਈ, ਇਹ Azure AI Foundry ਦੀ ਵਰਤੋਂ ਕਰਦੀ ਹੈ। ਆਪਣੇ ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਸੈਟਅੱਪ ਕਰਨ ਲਈ, ਤੁਸੀਂ [ਸੈਟਅੱਪ ਪਾਠ]( /00-course-setup/README.md) ਦੀ ਪਾਲਣਾ ਕਰ ਸਕਦੇ ਹੋ।


## ਲੋੜੀਂਦੇ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰੋ


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## ਐਮਸੀਪੀ ਪਲੱਗਇਨ ਕਨੈਕਸ਼ਨ ਬਣਾਉਣਾ

ਅਸੀਂ [OpenBnB MCP ਸਰਵਰ](https://github.com/openbnb-org/mcp-server-airbnb) ਨਾਲ MCPStdioPlugin ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਕਨੈਕਟ ਕਰਾਂਗੇ। ਇਹ ਸਰਵਰ @openbnb/mcp-server-airbnb ਪੈਕੇਜ ਰਾਹੀਂ Airbnb ਖੋਜ ਦੀ ਸਹੂਲਤ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ।


## ਕਲਾਇੰਟ ਬਣਾਉਣਾ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਆਪਣੇ LLM ਐਕਸੈੱਸ ਲਈ Azure AI Foundry ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਸਹੀ ਤਰੀਕੇ ਨਾਲ ਸੈਟ ਕੀਤੇ ਗਏ ਹਨ।


## ਵਾਤਾਵਰਣ ਸੰਰਚਨਾ

Azure OpenAI ਸੈਟਿੰਗਾਂ ਸੰਰਚਿਤ ਕਰੋ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਕੋਲ ਹੇਠਾਂ ਦਿੱਤੇ ਵਾਤਾਵਰਣ ਚਰਾਂ ਸੈਟ ਕੀਤੇ ਹੋਏ ਹਨ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਨੂੰ ਸਮਝਣਾ

ਇਹ ਨੋਟਬੁੱਕ **ਅਸਲੀ OpenBnB MCP ਸਰਵਰ** ਨਾਲ ਜੁੜਦੀ ਹੈ ਜੋ ਅਸਲੀ Airbnb ਖੋਜ ਦੀ ਸਹੂਲਤ ਪ੍ਰਦਾਨ ਕਰਦੀ ਹੈ।

### ਇਹ ਕਿਵੇਂ ਕੰਮ ਕਰਦਾ ਹੈ:

1. **MCPStdioPlugin**: MCP ਸਰਵਰ ਨਾਲ ਮਿਆਰੀ ਇਨਪੁਟ/ਆਉਟਪੁੱਟ ਸੰਚਾਰ ਦੀ ਵਰਤੋਂ ਕਰਦਾ ਹੈ
2. **ਅਸਲੀ NPM ਪੈਕੇਜ**: `@openbnb/mcp-server-airbnb` ਨੂੰ npx ਰਾਹੀਂ ਡਾਊਨਲੋਡ ਅਤੇ ਚਲਾਉਂਦਾ ਹੈ
3. **ਲਾਈਵ ਡਾਟਾ**: Airbnb ਦੇ APIs ਤੋਂ ਅਸਲੀ ਜਾਇਦਾਦਾਂ ਦੇ ਡਾਟਾ ਨੂੰ ਵਾਪਸ ਕਰਦਾ ਹੈ
4. **ਫੰਕਸ਼ਨ ਖੋਜ**: ਏਜੰਟ MCP ਸਰਵਰ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨਾਂ ਨੂੰ ਆਪਣੇ ਆਪ ਖੋਜ ਲੈਂਦਾ ਹੈ

### ਉਪਲਬਧ ਫੰਕਸ਼ਨ:

OpenBnB MCP ਸਰਵਰ ਆਮ ਤੌਰ 'ਤੇ ਇਹ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ:
- **search_listings** - ਸਥਾਨ ਅਤੇ ਮਾਪਦੰਡਾਂ ਦੁਆਰਾ Airbnb ਜਾਇਦਾਦਾਂ ਦੀ ਖੋਜ ਕਰੋ
- **get_listing_details** - ਖਾਸ ਜਾਇਦਾਦਾਂ ਬਾਰੇ ਵਿਸਥਾਰਿਤ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ
- **check_availability** - ਖਾਸ ਤਾਰੀਖਾਂ ਲਈ ਉਪਲਬਧਤਾ ਦੀ ਜਾਂਚ ਕਰੋ
- **get_reviews** - ਜਾਇਦਾਦਾਂ ਲਈ ਸਮੀਖਾਵਾਂ ਪ੍ਰਾਪਤ ਕਰੋ
- **get_host_info** - ਜਾਇਦਾਦ ਦੇ ਹੋਸਟਾਂ ਬਾਰੇ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ

### ਪੂਰਵ ਸ਼ਰਤਾਂ:

- **Node.js** ਤੁਹਾਡੇ ਸਿਸਟਮ 'ਤੇ ਇੰਸਟਾਲ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ
- **ਇੰਟਰਨੈਟ ਕਨੈਕਸ਼ਨ** MCP ਸਰਵਰ ਪੈਕੇਜ ਨੂੰ ਡਾਊਨਲੋਡ ਕਰਨ ਲਈ
- **NPX** ਉਪਲਬਧ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ (ਇਹ Node.js ਦੇ ਨਾਲ ਆਉਂਦਾ ਹੈ)

### ਕਨੈਕਸ਼ਨ ਦੀ ਜਾਂਚ:

ਤੁਸੀਂ MCP ਸਰਵਰ ਨੂੰ ਹੱਥੋਂ ਚਾਲੂ ਕਰਕੇ ਜਾਂਚ ਸਕਦੇ ਹੋ:
```bash
npx -y @openbnb/mcp-server-airbnb
```

ਇਹ OpenBnB MCP ਸਰਵਰ ਨੂੰ ਡਾਊਨਲੋਡ ਅਤੇ ਸ਼ੁਰੂ ਕਰੇਗਾ, ਜਿਸ ਨਾਲ Semantic Kernel ਅਸਲੀ Airbnb ਡਾਟਾ ਲਈ ਜੁੜਦਾ ਹੈ।


## ਓਪਨBnB MCP ਸਰਵਰ ਨਾਲ ਏਜੰਟ ਚਲਾਉਣਾ

ਹੁਣ ਅਸੀਂ AI ਏਜੰਟ ਚਲਾਉਣ ਜਾ ਰਹੇ ਹਾਂ ਜੋ ਓਪਨBnB MCP ਸਰਵਰ ਨਾਲ ਜੁੜਦਾ ਹੈ ਤਾਂ ਜੋ 2 ਵੱਡਿਆਂ ਅਤੇ 1 ਬੱਚੇ ਲਈ ਸਟਾਕਹੋਮ ਵਿੱਚ ਅਸਲੀ Airbnb ਰਿਹਾਇਸ਼ਾਂ ਦੀ ਖੋਜ ਕਰ ਸਕੇ। ਖੋਜ ਦੇ ਮਾਪਦੰਡਾਂ ਨੂੰ ਬਦਲਣ ਲਈ `user_inputs` ਲਿਸਟ ਨੂੰ ਬਦਲਣ ਲਈ ਬੇਝਿਜਕ ਹੋਵੋ।


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

ਸਾਰਾਂਸ਼  
ਵਧਾਈਆਂ! ਤੁਸੀਂ ਸਫਲਤਾਪੂਰਵਕ ਇੱਕ AI ਏਜੰਟ ਤਿਆਰ ਕੀਤਾ ਹੈ ਜੋ ਮਾਡਲ ਕਾਂਟੈਕਸਟ ਪ੍ਰੋਟੋਕੋਲ (MCP) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਸਲੀ ਦੁਨੀਆ ਦੇ ਰਿਹਾਇਸ਼ ਖੋਜ ਨਾਲ ਜੁੜਦਾ ਹੈ:

ਵਰਤੀਆਂ ਗਈਆਂ ਤਕਨਾਲੋਜੀਆਂ:  
ਸੈਮੈਂਟਿਕ ਕਰਨਲ - Azure OpenAI ਨਾਲ ਬੁੱਧੀਮਾਨ ਏਜੰਟ ਬਣਾਉਣ ਲਈ  
Azure AI Foundry - LLM ਸਮਰੱਥਾਵਾਂ ਅਤੇ ਚੈਟ ਪੂਰਨਤਾ ਲਈ  
MCP (ਮਾਡਲ ਕਾਂਟੈਕਸਟ ਪ੍ਰੋਟੋਕੋਲ) - ਸਟੈਂਡਰਡ ਟੂਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਲਈ  
OpenBnB MCP ਸਰਵਰ - ਅਸਲੀ Airbnb ਖੋਜ ਫੰਕਸ਼ਨਲਿਟੀ ਲਈ  
Node.js/NPX - ਬਾਹਰੀ MCP ਸਰਵਰ ਚਲਾਉਣ ਲਈ  

ਤੁਸੀਂ ਕੀ ਸਿੱਖਿਆ:  
MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ: ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਏਜੰਟਾਂ ਨੂੰ ਬਾਹਰੀ MCP ਸਰਵਰਾਂ ਨਾਲ ਜੋੜਨਾ  
ਰਿਅਲ-ਟਾਈਮ ਡਾਟਾ ਐਕਸੈਸ: ਲਾਈਵ APIਜ਼ ਰਾਹੀਂ ਅਸਲੀ Airbnb ਪ੍ਰਾਪਰਟੀਜ਼ ਦੀ ਖੋਜ  
ਪ੍ਰੋਟੋਕੋਲ ਕਮਿਊਨੀਕੇਸ਼ਨ: ਏਜੰਟ ਅਤੇ MCP ਸਰਵਰ ਦੇ ਵਿਚਕਾਰ stdio ਕਮਿਊਨੀਕੇਸ਼ਨ ਦੀ ਵਰਤੋਂ  
ਫੰਕਸ਼ਨ ਡਿਸਕਵਰੀ: MCP ਸਰਵਰਾਂ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨਾਂ ਦੀ ਆਟੋਮੈਟਿਕ ਖੋਜ  
ਸਟ੍ਰੀਮਿੰਗ ਰਿਸਪਾਂਸ: ਫੰਕਸ਼ਨ ਕਾਲਾਂ ਨੂੰ ਰਿਅਲ-ਟਾਈਮ ਵਿੱਚ ਕੈਪਚਰ ਅਤੇ ਲੌਗ ਕਰਨਾ  
HTML ਰੈਂਡਰਿੰਗ: ਸਜਾਏ ਹੋਏ ਟੇਬਲਾਂ ਅਤੇ ਇੰਟਰਐਕਟਿਵ ਡਿਸਪਲੇ ਨਾਲ ਏਜੰਟ ਦੇ ਜਵਾਬਾਂ ਨੂੰ ਫਾਰਮੈਟ ਕਰਨਾ  

ਅਗਲੇ ਕਦਮ:  
ਵਧੇਰੇ MCP ਸਰਵਰਾਂ (ਮੌਸਮ, ਉਡਾਣਾਂ, ਰੈਸਟੋਰੈਂਟ) ਨੂੰ ਸ਼ਾਮਲ ਕਰੋ  
MCP ਅਤੇ A2A ਪ੍ਰੋਟੋਕੋਲ ਨੂੰ ਜੋੜ ਕੇ ਇੱਕ ਮਲਟੀ-ਏਜੰਟ ਸਿਸਟਮ ਬਣਾਓ  
ਆਪਣੇ ਡਾਟਾ ਸਰੋਤਾਂ ਲਈ ਕਸਟਮ MCP ਸਰਵਰ ਬਣਾਓ  
ਸੈਸ਼ਨਾਂ ਵਿੱਚ ਲਗਾਤਾਰ ਗੱਲਬਾਤ ਦੀ ਯਾਦਸ਼ਕਤੀ ਲਾਗੂ ਕਰੋ  
MCP ਸਰਵਰ ਆਰਕੇਸਟਰੇਸ਼ਨ ਨਾਲ ਏਜੰਟ ਨੂੰ Azure Functions 'ਤੇ ਡਿਪਲੌਇ ਕਰੋ  
ਯੂਜ਼ਰ ਪ੍ਰਮਾਣਿਕਤਾ ਅਤੇ ਬੁਕਿੰਗ ਸਮਰੱਥਾਵਾਂ ਸ਼ਾਮਲ ਕਰੋ  

MCP ਆਰਕੀਟੈਕਚਰ ਦੇ ਮੁੱਖ ਫਾਇਦੇ:  
ਸਟੈਂਡਰਡਾਈਜ਼ੇਸ਼ਨ: ਬਾਹਰੀ ਟੂਲਾਂ ਨਾਲ AI ਏਜੰਟਾਂ ਨੂੰ ਜੋੜਨ ਲਈ ਯੂਨੀਵਰਸਲ ਪ੍ਰੋਟੋਕੋਲ  
ਰਿਅਲ-ਟਾਈਮ ਡਾਟਾ: ਵੱਖ-ਵੱਖ ਸੇਵਾਵਾਂ ਤੋਂ ਲਾਈਵ, ਅਪ-ਟੂ-ਡੇਟ ਜਾਣਕਾਰੀ ਤੱਕ ਪਹੁੰਚ  
ਵਿਸਤਾਰਯੋਗਤਾ: ਨਵੇਂ ਡਾਟਾ ਸਰੋਤਾਂ ਅਤੇ ਟੂਲਾਂ ਦੀ ਆਸਾਨ ਇੰਟੀਗ੍ਰੇਸ਼ਨ  
ਇੰਟਰਓਪਰੇਬਿਲਿਟੀ: ਵੱਖ-ਵੱਖ AI ਫਰੇਮਵਰਕ ਅਤੇ ਏਜੰਟ ਪਲੇਟਫਾਰਮਾਂ ਵਿੱਚ ਕੰਮ ਕਰਦਾ ਹੈ  
ਸੰਬੰਧਿਤ ਚਿੰਤਾਵਾਂ ਤੋਂ ਵੱਖਰਾ: AI ਲੌਜਿਕ ਅਤੇ ਬਾਹਰੀ ਡਾਟਾ ਐਕਸੈਸ ਦੇ ਵਿਚਕਾਰ ਸਪਸ਼ਟ ਵੰਡ  



---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਣਭਵਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੇ ਪ੍ਰਯੋਗ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਮੀਆਂ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆਵਾਂ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
